---
title: Think about the flow
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.2
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
---

1. Make API calls onto 2024
1. Include date of data insert into API data
1. Insert 2024 data where date of data insert == today into MongoDB
1. Delete 2024 data where date of data insert != today

**Lets see if I can shift this into polars? Cos I may be dealing with a very large dataset**


In [2]:
import os
import json
import requests
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import plotly.graph_objects as go

### Calculate 6 months before period 

In [20]:
latest_df.columns

Index(['month', 'town', 'flat', 'block', 'street_name', 'storey_range', 'area',
       'lease_left_mths', 'price'],
      dtype='object')

In [18]:
current_mth = datetime.now().date().strftime("%Y-%m")
total_periods = [str(i)[:7] for i in pd.date_range(
    "2023-01-01", current_mth + "-01", freq='MS').tolist()]
recent_periods = total_periods[-6:]

df_cols = ['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 'floor_area_sqm', 'remaining_lease', 'resale_price']
param_fields = ",".join(df_cols)
base_url = "https://data.gov.sg/api/action/datastore_search?resource_id="
ext_url = "d_8b84c4ee58e3cfc0ece0d773c8ca6abc"
full_url = base_url + ext_url

recent_df = pd.DataFrame()
for period in tqdm(recent_periods):    
    params = {
        "fields": param_fields,
        "filters": json.dumps({'month': period}),
        "limit": 10000
    }
    response = requests.get(full_url, params=params)
    mth_df = pd.DataFrame(response.json().get("result").get("records"))
    recent_df = pd.concat([recent_df, mth_df], axis=0)

latest_df = recent_df.copy()
latest_df.columns = ['month', 'town', 'flat', 'block', 'street_name', 'storey_range', 'area', 'lease_left', 'price']
latest_df['lease_left'] = [i.replace('s', '') for i in latest_df['lease_left']]
latest_df['lease_left'] = [i.replace(' year', 'y') for i in latest_df['lease_left']]
latest_df['lease_left'] = [i.replace(' month', 'm') for i in latest_df['lease_left']]
latest_df['lease_left_yrs'] = [int(i[0])*12 for i in latest_df['lease_left'].str.split("y")]
latest_df['lease_left_mths'] = [0 if i[-1].strip().replace('m', '') == '' else int(i[-1].strip().replace('m', '')) for i in latest_df['lease_left'].str.split("y")]
latest_df['total_lease_left_mths'] = latest_df['lease_left_yrs'] + latest_df['lease_left_mths']
del latest_df['lease_left']
del latest_df['lease_left_yrs']
del latest_df['lease_left_mths']
latest_df.rename(columns={'total_lease_left_mths':'lease_left_mths'}, inplace=True)

latest_df['storey_range'] = [i.replace(' TO ', '-') for i in latest_df['storey_range']]
latest_df['flat'] = [i.replace(" ROOM", "R") for i in latest_df['flat']]
latest_df['flat'] = [i.replace("EXECUTIVE", "EC") for i in latest_df['flat']]
latest_df['flat'] = [i.replace("MULTI-GENERATION", "MG") for i in latest_df['flat']]

latest_df['flat'] = latest_df['flat'].astype(str)
latest_df['lease_left_mths'] = latest_df['lease_left_mths'].astype(np.int32)
latest_df['area'] = latest_df['area'].astype(np.float16)
latest_df['price'] = latest_df['price'].astype(np.float32)

latest_df = latest_df[['month', 'town', 'flat', 'block', 'street_name', 'storey_range', 'area', 'lease_left_mths', 'price']]

### One Time set of API calls and data dump into MongoDB
#### Time parameters 

In [122]:
current_mth = datetime.now().date().strftime("%Y-%m")
total_periods = [str(i)[:7] for i in pd.date_range(
    "2014-01-01", current_mth+"-01", freq='MS').tolist()]

In [123]:
# df_cols = ['month', 'town', 'floor_area_sqm',
#            'flat_type', 'lease_commence_date', 'resale_price']

# Focus on most minimium cols for speed
df_cols = ['month', 'town', 'resale_price']
param_fields = ",".join(df_cols)
base_url = "https://data.gov.sg/api/action/datastore_search?resource_id="
latest_df = pd.DataFrame()

for period in tqdm(total_periods):
    if "2014-" in period:
        print(period)
        url = base_url + "d_2d5ff9ea31397b66239f245f57751537"
    elif "2015-" in period or "2016-" in period:
        url = base_url + "d_ea9ed51da2787afaf8e51f827c304208"
    else:
        url = base_url + "d_8b84c4ee58e3cfc0ece0d773c8ca6abc"
    
    params = {
        "fields": param_fields,
        "filters": json.dumps({'month': period}),
        "limit": 10000
    }
    response = requests.get(url, params=params)
    mth_df = pd.DataFrame(response.json().get("result").get("records"))
    latest_df = pd.concat([latest_df, mth_df], axis=0)

  0%|          | 0/127 [00:00<?, ?it/s]

2014-01
2014-02
2014-03
2014-04
2014-05
2014-06
2014-07
2014-08
2014-09
2014-10
2014-11
2014-12


### Updating just certain periods 

In [8]:
latest_df = pd.DataFrame()

df_cols = ["month", "town", "flat_type", "block", "street_name", "storey_range", "floor_area_sqm", "remaining_lease", "resale_price"]
today = datetime.now().date()
mth_first_d = today.replace(day=1)
last_mth = mth_first_d - timedelta(days=1)
last_mth, current_mth = last_mth.strftime("%Y-%m"), today.strftime("%Y-%m")

# All months
mths_2024_onwards = [str(i)[:7] for i in pd.date_range(
    "2024-01-01", current_mth + "-01", freq='MS').tolist()]

# Update recent two months
update_mths = [last_mth, current_mth]

param_fields = ",".join(df_cols)
basic_df = pd.DataFrame()

y2024 = "d_8b84c4ee58e3cfc0ece0d773c8ca6abc"
base_url = "https://data.gov.sg/api/action/datastore_search?resource_id="
url = base_url + y2024

for mth in tqdm(mths_2024_onwards):
    params = {
        "fields": param_fields,
        "filters": json.dumps({'month': mth}),
        "limit": 10000
    }
    response = requests.get(url, params=params)
    mth_df = pd.DataFrame(response.json().get("result").get("records"))   
    latest_df = pd.concat([latest_df, mth_df], axis=0)

  0%|          | 0/7 [00:00<?, ?it/s]

#### Past data 

In [105]:
latest_df = basic_df.copy()
latest_df = latest_df[['month', 'town', 'flat_type', 'block', 'street_name', 'storey_range', 
                       'floor_area_sqm', 'remaining_lease', 'resale_price', 'insert_date']]

latest_df.columns = ['month', 'town', 'flat', 'block', 'street_name', 'storey_range', 'area', 'lease_left', 'price', "insert_date"]
# latest_df['month'] = pd.to_datetime(latest_df['month'])
latest_df['lease_left'] = [i.replace('s', '') for i in latest_df['lease_left']]
latest_df['lease_left'] = [i.replace(' year', 'y') for i in latest_df['lease_left']]
latest_df['lease_left'] = [i.replace(' month', 'm') for i in latest_df['lease_left']]
latest_df['lease_left_yrs'] = [int(i[0])*12 for i in latest_df['lease_left'].str.split("y")]
latest_df['lease_left_mths'] = [0 if i[-1].strip().replace('m', '') == '' else int(i[-1].strip().replace('m', '')) for i in latest_df['lease_left'].str.split("y")]
latest_df['total_lease_left_mths'] = latest_df['lease_left_yrs'] + latest_df['lease_left_mths']
del latest_df['lease_left']
del latest_df['lease_left_yrs']
del latest_df['lease_left_mths']
latest_df.rename(columns={'total_lease_left_mths':'lease_left_mths'}, inplace=True)

latest_df['storey_range'] = [i.replace(' TO ', '-') for i in latest_df['storey_range']]
latest_df['flat'] = [i.replace(" ROOM", "R") for i in latest_df['flat']]
latest_df['flat'] = [i.replace("EXECUTIVE", "EC") for i in latest_df['flat']]
latest_df['flat'] = [i.replace("MULTI-GENERATION", "MG") for i in latest_df['flat']]

latest_df['flat'] = latest_df['flat'].astype(str)
latest_df['insert_date'] = latest_df['insert_date'].astype(str)
latest_df['lease_left_mths'] = latest_df['lease_left_mths'].astype(np.int32)
latest_df['area'] = latest_df['area'].astype(np.float16)
latest_df['price'] = latest_df['price'].astype(np.float32)

latest_df = latest_df[['month', 'town', 'flat', 'block', 'street_name', 'storey_range', 
                       'area', 'lease_left_mths', 'price', 'insert_date']]

In [106]:
latest_df.head()

,month,town,flat,block,street_name,storey_range,area,lease_left_mths,price,insert_date
0,2020-01,ANG MO KIO,3R,208,ANG MO KIO AVE 1,04-06,73.0,667,265000.0,2024-07-18
1,2020-01,ANG MO KIO,3R,307C,ANG MO KIO AVE 1,19-21,70.0,1100,470000.0,2024-07-18
2,2020-01,ANG MO KIO,3R,319,ANG MO KIO AVE 1,01-03,73.0,676,230000.0,2024-07-18
3,2020-01,ANG MO KIO,3R,216,ANG MO KIO AVE 1,04-06,73.0,663,280000.0,2024-07-18
4,2020-01,ANG MO KIO,3R,556,ANG MO KIO AVE 10,07-09,68.0,709,220000.0,2024-07-18


#### MongoDB Upload 

In [131]:
from pymongo import mongo_client
from typing import Dict, List
from datetime import datetime
import time
import os

# MongoDB credentials
MONGO_PASSWORD = os.environ["mongo_pw"]
base_url = "mongodb+srv://cliffchew84:"
end_url = "cliff-nlb.t0whddv.mongodb.net/?retryWrites=true&w=majority"
mongo_url = f"{base_url}{MONGO_PASSWORD}@{end_url}"

def connect_mdb():
    return mongo_client.MongoClient(mongo_url, serverSelectionTimeoutMS=5000)

db = connect_mdb()
db.list_database_names()

['nlb', 'admin', 'local']

In [132]:
# Checking DB
db_nlb = db['nlb']

# Check the collections inside the BD
db_nlb.list_collection_names()

['users',
 'user_status',
 'user_books',
 'user_search',
 'books_avail',
 'books_info',
 'p_att',
 'p_mrt']

#### Dump HDB data into MongoDB 

In [136]:
hdb_hist = db_nlb['hdb_latest']

# Convert DataFrame to dictionary
data_dict = latest_df.to_dict("records")

# Insert data into MongoDB
hdb_hist.insert_many(data_dict)

#### Test query from MongoDB 

In [111]:
query_output = db_nlb['hdb_latest'].find({}, {})
documents_list = list(query_output)
pd.DataFrame(documents_list)

,_id,month,town,flat,block,street_name,storey_range,area,lease_left_mths,price,insert_date
0,6698d67f9920c8136bde3735,2020-01,ANG MO KIO,3R,208,ANG MO KIO AVE 1,04-06,73.0,667,265000.0,2024-07-18
1,6698d67f9920c8136bde3736,2020-01,ANG MO KIO,3R,307C,ANG MO KIO AVE 1,19-21,70.0,1100,470000.0,2024-07-18
2,6698d67f9920c8136bde3737,2020-01,ANG MO KIO,3R,319,ANG MO KIO AVE 1,01-03,73.0,676,230000.0,2024-07-18
3,6698d67f9920c8136bde3738,2020-01,ANG MO KIO,3R,216,ANG MO KIO AVE 1,04-06,73.0,663,280000.0,2024-07-18
4,6698d67f9920c8136bde3739,2020-01,ANG MO KIO,3R,556,ANG MO KIO AVE 10,07-09,68.0,709,220000.0,2024-07-18
...,...,...,...,...,...,...,...,...,...,...,...
120471,6698d6809920c8136be00dcc,2024-07,YISHUN,EC,352,YISHUN RING RD,04-06,146.0,757,865000.0,2024-07-18
120472,6698d6809920c8136be00dcd,2024-07,YISHUN,EC,359,YISHUN RING RD,10-12,145.0,757,880000.0,2024-07-18
120473,6698d6809920c8136be00dce,2024-07,YISHUN,EC,277,YISHUN ST 22,01-03,146.0,723,770000.0,2024-07-18
120474,6698d6809920c8136be00dcf,2024-07,YISHUN,EC,277,YISHUN ST 22,04-06,146.0,722,800000.0,2024-07-18


### Query 

In [66]:
def create_mdb_query_w_df_cols(df: pd.DataFrame):
    """ Creates mongoDB query from dataframe or list of known column names"""

    if type(df) == pd.DataFrame:
        col_names = df.columns.tolist()
    elif type(df) == list:
        col_names = df
    elif type(df) == str:
        col_names = [i.strip() for i in df.split(',')]
    
    col_dict = dict()
    col_filter = {"_id" : 0}
    
    for name in col_names:
        col_dict[name] = {"$exists": True}
        col_filter[name] = 1
    
    return col_dict, col_filter

In [67]:
# Testing query on just a list of column names
mdb_query = create_mdb_query_w_df_cols(['month', 'town', 'price'])
query_p, query_f = mdb_query

In [68]:
list(db_nlb['hdb_updated'].find({ "month": {"$gt": "2011-12-01"}, "month": {"$lt": "2024-01-01" }}, query_f).limit(10))

[]

In [32]:
db_nlb["hdb_updated"].create_index("month")
db_nlb["hdb_updated"].create_index("town")
db_nlb["hdb_updated"].create_index("flat")

'flat_1'

In [35]:
query_output = db_nlb['hdb_updated'].find(query_p, query_f)
documents_list = list(query_output)
df = pd.DataFrame(documents_list)
df.shape

### Inject older records 

In [145]:
make_old_df = latest_df.head(100)
make_old_df.shape

(100, 10)

In [149]:
make_old_df['insert_date'] = '2024-07-17'
make_old_df.head(1)

,month,town,flat,block,street_name,storey_range,area,lease_left_mths,price,insert_date
0,2020-01,ANG MO KIO,3R,208,ANG MO KIO AVE 1,04-06,73.0,667,265000.0,2024-07-17


In [150]:
data_dict = make_old_df.to_dict("records")

# Insert data into MongoDB
hdb_updated.insert_many(data_dict)

### Testing filtering for latest records
- See if I can change this into a Polars extraction instead?

In [12]:
mdb_query = create_mdb_query_w_df_cols(latest_df)
query_p, query_f = mdb_query

query_output = db_nlb['hdb_updated'].find(query_p, query_f)
documents_list = list(query_output)
df = pd.DataFrame(documents_list)

NameError: name 'latest_df' is not defined

In [152]:
cols = df.columns[:-1].tolist()
df_results = df.sort_values('insert_date', ascending=False).groupby(cols).head(1)
df_results.shape

(120387, 10)

In [154]:
df_results.insert_date.value_counts()

insert_date
2024-07-18    120387
Name: count, dtype: int64